# **Sale Price Correlation Study**

## Objectives

1. Answer business requirement 1:
    * *The client is interested in discovering how the house attributes correlate with the sale price. Therefore, the client expects data visualizations of the correlated variables against the sale price to show that.*
    * In this notebook, we will perform a correlation study and plot graphs to represent our findings


2. Attempt to validate our hypotheses:
    * As laid out in the corresponding REAME.md, our correltation sudy can be used to test our project hypotheses

## Inputs

* inputs/datasets/unzipped/house_prices_records.csv 

## Outputs

* In this notebook we will generate code and graphs to answer buiness requiremnt 1, test our hypotheses and use in our Streamlit App


## CRISP-DM
* Data Understanding

---

# Change working directory

* The notebooks are stored in a sub folder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")    

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Load Data

In [ ]:
import pandas as pd
df = pd.read_csv(f"inputs/datasets/unzipped/house_prices_records.csv")
print(df.shape)
df.head()

In [ ]:
df.info()

* We will run a profile report in order to get a more detailed look at our data

In [ ]:
from pandas_profiling import ProfileReport

pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

---

# Correlation Study

* Before we check the correlation rates of our variables against the Sale Price, we will use One Hot Encoding on our categorical variables.
* In order to use our correlation methods, our categorical variables need to be numbers.  One Hot Encoding will allow us to easily see the original values as they split the categorical variables into new columns and assign them a binary value of '1' or '0'.
* To carry out One Hot Encoding, we first need to handle the missing variable in our categorical columns and as we can see above in our dataset, there are a number of columns with missing data. 
* Let's check which columns with missing data below:

In [ ]:
vars_with_missing_data = df[df.columns[df.isna().sum() > 0]]
vars_with_missing_data

* We will take a list of the categorical variables from this DataFrame to use with our One Hot Encoder and will impute the most frequent value for any missing values for each column.

In [ ]:
cat_vars_with_na = (vars_with_missing_data
                    .columns[vars_with_missing_data.dtypes == 'object']
                    .to_list())
cat_vars_with_na

* We impute the missing values with CategoricalImputer

In [ ]:
from feature_engine.imputation import CategoricalImputer
categorical_imputer = CategoricalImputer(imputation_method='frequent',
                                         variables=cat_vars_with_na)
df = categorical_imputer.fit_transform(df)

* Let's check our DataSet to ensure there is no missing values for ['BsmtFinType1', 'GarageFinish']

In [ ]:
df.filter(cat_vars_with_na).info()

* We use OneHotEncoder and declare a new DataFrame with our new columns

In [ ]:
from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=cat_vars_with_na, drop_last=False)
df_ohe = encoder.fit_transform(df)
print(df_ohe.shape)
df_ohe.head(3)

---

* We will use both person and spearman methods to check the levels of correlation between variables and 'SalePrice'. We will exclude 'SalePrice' from our results and check the 5 most highly correlated varibales for each method.

In [ ]:
df_pearson = (df_ohe.corr(method='pearson')['SalePrice']
             .sort_values(key=abs, ascending=False)[1:].head(5))
df_pearson

In [ ]:
df_spearman = (df_ohe.corr(method='spearman')['SalePrice']
              .sort_values(key=abs, ascending=False)[1:].head(5))
df_spearman

* The methods produce similar enough results.
* 'OverallQual' is the most highly correlated variable in each case and only 'YearBuilt' & '1stFlrSF' do not show up in each
* Let's list out the variables to study in a set to remove any duplicates

In [ ]:
cols_to_study = set(df_spearman.index.to_list()+df_pearson.index.to_list())
cols_to_study

* We create a new DataFrame with the variables most closely correlated with Sales Price

In [ ]:
df_eda = df_ohe.filter(['1stFlrSF', 'GarageArea', 'GrLivArea', 'OverallQual',
                        'TotalBsmtSF', 'YearBuilt', 'SalePrice'])
df_eda.head()

# Data Visualisation
* We now want to plot out our data in order to show how these variables correlate to Sales Prices.
* 'SalePrice' as a variable has a large range of values

In [ ]:
min_val = int(df_eda.filter(['SalePrice']).min())
max_val = int(df_eda.filter(['SalePrice']).max())
count = df_eda.shape[0]

print(f"There are {count} values in the SalePrice column ranging from" +
      " {min_val} to {max_val}")

* For this reason, to simplify the visiualisation of our Sale Prices, we will seperate the values into 6 quantiles which will represent ranges of values and display more clearly on histograms.
* We will do so using EqualFrequencyDiscretiser.
* We fit and transform our data below:


In [ ]:
from feature_engine.discretisation import EqualFrequencyDiscretiser
discretiser = EqualFrequencyDiscretiser(q=6, variables=['SalePrice'])
discretiser.fit(df_eda)
df_eda = discretiser.transform(df_eda)
df_eda

* We confirm our 'SalePrice' variable has been split up into equal quantiles

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_theme(style="whitegrid")
sns.histplot(data=df_eda, x='SalePrice')
plt.show()

* This has worked, however it is not clear what range the values of 0 - 5 represent.
* We will check how the range of values has been discretised

In [ ]:
discretiser.binner_dict_

* We will assign these ranges to the values they represent in a dictionary

In [ ]:
labels = discretiser.binner_dict_['SalePrice']
q_value = len(labels)-1
labels_map = {}

for x in range(0, q_value):
    if x == 0:
        labels_map[x] = f"< {int(labels[1])}"
    elif x < q_value - 1:
        labels_map[x] = f"{int(labels[x])} - {int(labels[x+1])}"
    else:
        labels_map[x] = f"{int(labels[x])} +"

labels_map

* We apply these values to our DataFrame

In [ ]:
df_eda["SalePrice"] = df_eda["SalePrice"].replace(labels_map)
df_eda

* We define the hue_order for our histograms

In [ ]:
hue_order = labels_map.values()
list(hue_order)

* We will plot our correlated variables on histograms below.
    * plot_numerical() function adapted from CI Lesson: *Exploratory Data Analysis > Exploratory Data Analysis Tools > Correlation Unit 2: Analysis*
* We will also plot accompanying lineplots on the original dataset

In [ ]:
%matplotlib inline
import numpy as np
sns.set_style('whitegrid')


def plot_numerical(df, col, target_var, hue_order):
    plt.figure(figsize=(8, 5))
    sns.histplot(data=df, x=col, hue=target_var, hue_order=hue_order, kde=True,
                 element="step")
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()


def plot_line(df, col, target_var):
    plt.figure(figsize=(8, 5))
    sns.lineplot(data=df, x=target_var, y=col, color='#FC6C85')
    plt.title(f"{col}", fontsize=20, y=1.05)
    plt.show()


target_var = 'SalePrice'
for col in ['1stFlrSF', 'GarageArea', 'GrLivArea', 'OverallQual',
            'TotalBsmtSF', 'YearBuilt']:
    plot_numerical(df_eda, col, target_var, hue_order)
    plot_line(df, col, target_var)
    print("\n\n")


## Conclusions
* Houses with high Sales Prices tend to have first floors with at least 1500 square feet.

* Houses with low Sales Prices tend to have no garage and those with a garage of at least 600 square feet tend to have high Sales Prices.

* Houses with high sales prices tend to have above grade living area of at least 1500 square feet. Those with low sales prices tend to have 1000 square feet or less.

* Houses with high Sales prices tend to have at least a Very Good Overall Quality Rating.

* Houses with high Sales Prices tend to have basements with at a square footage of at least 1200. Houses with no basements or basements with less than 1000 square feet tend to have low Sales prices.

* Houses do not tend to have a high Sales Price if built before 1990.


# Testing our Hypotheses

* Next, as part of Sale Price Study, we will test our hypotheses and will do so using correlation studies:

## **1. Larger houses will tend to have sell at a higher price.**

* We will test this by checking whether properties with higher values for '1stFlrSF' tend to have higher Sales Prices

* If we suspect that properties with greater 1st Floor square footage will generally have a higher sales price, then we could expect a linear correlation between the variables '1stFlrSF' & 'SalePrice'.
* Therefore, we will check the pearson correlation levels between these variables:

In [ ]:
df_pearson = df.corr(method='pearson')['SalePrice'].filter(['1stFlrSF'])
df_pearson

* As we can see, the level of pearson correlation between the varaibles is 0.61, which is relatively high.
* This does not necessarily confirm that larger properties have higher sales prices, but we can use a scatterplot to visualise the data and gleam a better understanding

In [ ]:
x, y = '1stFlrSF', 'SalePrice'
fig, axes = plt.subplots(figsize=(12, 6))
sns.scatterplot(data=df, x=x, y=y)
plt.show()

* We can see there is a general trend in higher SalePrice values when there is higher 1st Floor Square Footage.
* It is not hugely pronounced, hwoever, we can note that there are no houses worth less than $100,000 where the '1stFlrSF' value is higher than 1500.
* We can also examine the y-axis and note that there are no Sale Prices greater than $300,000 where the '1stFlrSF' value is less than 1000.
* If we examine our histogram from our previous study, we can see that houses in the higher range of Sale Prices tend to have at least 1500 first floor square footage

In [ ]:
plot_numerical(df_eda, '1stFlrSF', 'SalePrice', hue_order)

* Our correlation sutdy and visualisations support our hypothesis

## **2. The overall condition of the house will result in a higher sale price.**
* We will check how OverallCond ratings correlated to SalePrice and also check whether homes with similar values for '1stFlrSF', with greater OverallCond, tend to have greater Sale prices.

* Similar to our first hypothesis, we should suspect a that the OverallCond might correlate linearly with the SalePrice if a higher rating is to result in a higher Sale price.
* We will check the level of pearson correlation bewteen the variables.

In [ ]:
df_pearson = df.corr(method='pearson')['SalePrice'].filter(['OverallCond'])
df_pearson

* The level of linear correlation between the 'OverallCond' & 'SalePrice' is 0.08, which is very low and does not suggest that the overall condition rating is a strong factor in the Sale Price.
* This does not support our hypothesis, however, there are numerous variables which affect the Sale Price, so it may be the case that the Overall Condiiton rating of a house may play a larger role when taking houses of similar size into account. This forms part of our hypothesis, so we will try and validate this.
* To do so, we willneed to group houses of similar 1st floor square footage together, so we will discretize the column values and convert the variable to a categorical variable.
* We will use EqualFrequencyDiscretiser to do so but also Outlier Trimmer in order to have more accurate ranges of '1stFlrSF'.

#### Outlier Trimmer

In [ ]:
from feature_engine.outliers import OutlierTrimmer
df_hyp = df.copy()
outlier_trimmer = OutlierTrimmer(capping_method='iqr', fold=1.5, tail='both',
                                 variables=['1stFlrSF'])
df_hyp = outlier_trimmer.fit_transform(df_hyp)

#### Discretisation

In [ ]:
discretiser = EqualFrequencyDiscretiser(q=6, variables=['1stFlrSF'])
discretiser.fit(df_hyp)
df_hyp = discretiser.transform(df_hyp)
df_hyp.filter(['1stFlrSF']).head(5)

* Confirm the range of values has been split into 4 categories with equal counts

In [ ]:
sns.set_theme(style="whitegrid")
sns.histplot(data=df_hyp, x='1stFlrSF')
plt.show()

* We check how the range of values has been discretised

In [ ]:
discretiser.binner_dict_

* We assign these ranges to the values which they represent

In [ ]:
labels = discretiser.binner_dict_['1stFlrSF']
q_value = len(labels)-1
labels_map = {}

for x in range(0, q_value):
    if x == 0:
        labels_map[x] = f"< {int(labels[1])}"
    elif x < q_value - 1:
        labels_map[x] = f"{int(labels[x])} - {int(labels[x+1])}"
    else:
        labels_map[x] = f"{int(labels[x])} +"

labels_map

* We apply these to df_hyp

In [ ]:
df_hyp['1stFlrSF'] = df_hyp['1stFlrSF'].replace(labels_map)
df_hyp.head(5)

In [ ]:
order = labels_map.values()
list(order)

* We will now visualise '1stFlrSF' against 'SalePrice' and use a hue to represent the 'OverallCond' ratings.

In [ ]:
x, y = '1stFlrSF', 'SalePrice'
fig, axes = plt.subplots(figsize=(12, 6))
sns.stripplot(data=df_hyp, x=x, y=y, hue='OverallCond', order=order)
plt.show()

* From looking at our strip plot, we can see that generally the most frequent rating for the most expensive properties for each '1stFlrSF' range is 5, which is not completely surprising as it is the most frequent rating in our dataset.
* We can also see that any property rated 2 or below tend to be towards the bottom in terms of Sale price.
* It is not totally clear from our plot, that a higher rating will translate to a higher Sale Price for similarly sized properties:
    * Some 9 rated properties can be seen at the top of some ranges but looking at 1516+ for example, there are properties rated 9 which are towards the bottom.
    * If we look throughout each range and only take ratings of 7+ into account, they seem to be spread accross the range of Sale Prices for each quantile of 1t floor square foot.
* With these arguments and the low level of correlation studied earlier, our correlation study does not validate our hypothesis, but rather contradicts it.

## **3. Houses with Garages will tend to have a higher sales price than those without and those with large garages will tend to sell at higher prices.**

* We again will test this hypothesis using a correlation study and visualisations.
* We would again suspect that the GarageArea might correlate linearly with the SalePrice if a larger garage is to result in a higher Sale price.
* We will check the level of pearson correlation bewteen the variables.

In [ ]:
df_pearson = df.corr(method='pearson')['SalePrice'].filter(['GarageArea'])
df_pearson

* This is a relatively high score. It appears that the size of the Garage will likely lead to a higher sales price.
* We will plot the data out and check that it confirms this idea.
* We can use the plot from our previous correlation study and inspect.

In [ ]:
plot_numerical(df_eda, 'GarageArea', 'SalePrice', hue_order)

* It seems clear from our histogram that the trend of larger garages results in a higher sale price. 
    * The higher range of Sale Prices are concentrated in the higher end of the scale when it comes to Garage Area and apart from a few outliers, almost all properties with the largest garages have high sale prices.


* It is also clear from the high number of properties with sale prices of less than $118,500, that properties without garages ('GarageArea'=0) will tend to have a lower sale price than those that do.
    * There are come properties sold without garages in the higher range of sale prices, but these are a minority.
    * It can also be seen, that outside of properties with no garage, the bulk of properties with sale prices of less than $118,500, have garages which fall between 200 and 400 square foot.

* Our correlation study in this case validates our hypothesis.

---

# Next Steps

* We have completed our first business requirment and generated content for our dashboard.
* The next action is to move on to business requirment 2 and build an ML pipeline to predict the sale price of the inherited homes and any other homes in Ames, Iowa.
* There are no files to push to repo from this notebook.